<a href="https://colab.research.google.com/github/Xage-masa/GitInstructionWorkshop/blob/main/Foursquare_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Урок 1. Foursquare API: Расширенный поиск заведений по категории и ключевому слову

import requests
import time

# Вставленный API-ключ
FOURSQUARE_API_KEY = "fsq3Tejzr0YuKPig/dK8KWJInDjmqn7gZ296E9r12humvOA="
headers = {
    "Accept": "application/json",
    "Authorization": FOURSQUARE_API_KEY
}

# Пользовательский ввод
category = input("Введите категорию (например, кофейни, музеи, парки): ")
location = input("Введите город (например, Москва, Санкт-Петербург): ")

# Геолокация через OpenStreetMap
geocode_url = f"https://nominatim.openstreetmap.org/search?q={location}&format=json"
geo_headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(geocode_url, headers=geo_headers)

if not response.ok or not response.json():
    print("Не удалось определить координаты города. Проверьте название.")
else:
    coords = response.json()[0]
    lat = coords['lat']
    lon = coords['lon']

    # Словарь популярных категорий Foursquare
    fsq_categories = {
        "кофейни": "13032",  # Coffee Shop
        "музеи": "13027",    # Museum
        "парки": "16032",    # Park
        "рестораны": "13065" # Restaurant
    }

    fsq_cat = fsq_categories.get(category.lower(), None)

    url = "https://api.foursquare.com/v3/places/search"
    params = {
        "ll": f"{lat},{lon}",
        "limit": 20,
        "radius": 10000,
        "sort": "RELEVANCE"
    }
    if fsq_cat:
        params["categories"] = fsq_cat
    else:
        params["query"] = category

    print(f"\nЗаведения по запросу '{category}' в городе '{location}':\n")
    res = requests.get(url, headers=headers, params=params)

    if res.ok:
        data = res.json().get("results", [])
        if not data:
            print("Ничего не найдено.")
        else:
            for i, place in enumerate(data, 1):
                fsq_id = place.get("fsq_id")
                name = place.get("name", "Без названия")
                location_info = place.get("location", {})
                address = location_info.get("formatted_address")
                if isinstance(address, list):
                    address = ", ".join(address)
                elif not address:
                    address = "Нет данных"

                # Запрос для рейтинга
                details_url = f"https://api.foursquare.com/v3/places/{fsq_id}?fields=rating"
                details_res = requests.get(details_url, headers=headers)
                rating = "Рейтинг не указан"
                if details_res.ok:
                    rating_val = details_res.json().get("rating")
                    if rating_val:
                        rating = str(rating_val)

                print(f"{i}. {name}\n   Адрес: {address}\n   Рейтинг: {rating}\n")
                time.sleep(0.3)
    else:
        print("Ошибка при запросе к Foursquare API.")


Введите категорию (например, кофейни, музеи, парки): кофейни
Введите город (например, Москва, Санкт-Петербург): Санкт-Петербург

Заведения по запросу 'кофейни' в городе 'Санкт-Петербург':

1. CoffeeCube
   Адрес: Савушкина 128, 197374, Санкт-Петербург
   Рейтинг: 8.2

2. Coffee Point
   Адрес: 199155, Санкт-Петербург
   Рейтинг: 8.3

3. Кинг Понг
   Адрес: Савушкина Ул., д. 119, 197374, Санкт-Петербург
   Рейтинг: 8.2

4. Sibaristica
   Адрес: наб. Обводного канала, 199, 190020, Санкт-Петербург
   Рейтинг: 9.2

5. Mntn Coffee
   Адрес: Большая Зеленина ул., 34, 197110, Санкт-Петербург
   Рейтинг: 8.5

6. Микрокофейня Тчк
   Адрес: Каменноостровский Пр., 18/11 (ул. Мира), 197046, Санкт-Петербург
   Рейтинг: 8.9

7. Буше
   Адрес: Средний В.О. Пр., 33, 199004, Санкт-Петербург
   Рейтинг: 8.3

8. Кофейня Colors
   Адрес: Ленина Ул., д. 27, 197136, Санкт-Петербург
   Рейтинг: 8.5

9. Кофейня Будем знакомы
   Адрес: Малый Пр., П. С., 60/17, 197136, Санкт-Петербург
   Рейтинг: 8.4

10. Кофей